# Collaborative Filtering Recommendations using Original Values

In [1]:
import pandas as pd
import numpy as np
import implicit

from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix

In [2]:
train = pd.read_csv("data\\train-plays.csv")
test = pd.read_csv("data\\test-plays.csv")
game_coding = pd.read_csv("data\\game-coding.csv")

## Format training data

In [3]:
game_user = train.pivot(index="game_id", columns="user_id", values="amount")

In [4]:
game_user = game_user.fillna(0)

In [5]:
game_user_sparse = csr_matrix(game_user)

In [6]:
user_game = game_user.T

In [7]:
user_game_sparse = csr_matrix(user_game)

In [8]:
user_ids = train['user_id'].unique()

## Alternating Least Squares

In [9]:
als = implicit.als.AlternatingLeastSquares(128)

In [10]:
als.fit(game_user_sparse)

100%|██████████| 15/15 [00:00<00:00, 30.74it/s]


In [11]:
rows_list = list()
games_recommended = set()
for user_id in user_ids:
    results_dict = dict()
    results_dict['user_id'] = user_id
    test_df = test[test['user_id'] == user_id]
    recommendations_df = pd.DataFrame(als.recommend(user_id, user_game_sparse))
    recommended = test_df.merge(recommendations_df, left_on="game_id", right_on=0)
    results_dict['num_recommended'] = recommended.shape[0]
    results_dict['total_to_recommend'] = test_df.shape[0]
    rows_list.append(results_dict)
    games_recommended.update(recommended["game_id"].unique())

In [12]:
results = pd.DataFrame(rows_list)

In [13]:
results['num_recommended'].sum() / results['total_to_recommend'].sum()

0.16936125099390406

In [14]:
len(games_recommended) / train['game_id'].unique().size

0.19252691576947434